## Model Name
MultI-Task  Ordinal  Regression  (MITOR)  
Spatial  In-complete Multi-task Deep leArning (SIMDA)  
Pending Task: Deciding the name

In [1]:
# Reading Data
import scipy.io
import utils
import configure as config
# data processing
config.write_params()
data = scipy.io.loadmat(config.data_location) # data format: dict
[X_train, Y_train, X_test, Y_test, adj_data] = utils.data_formulate(data)

C:\ProgramData\Anaconda3\envs\Incomplete_label_multi_task\lib\site-packages\scipy\io\matlab\mio.py:226: MatReadWarning: Duplicate variable name "None" in stream - replacing previous with new
Consider mio5.varmats_from_mat to split file into single variable files
  matfile_dict = MR.get_variables(variable_names)


In [2]:
import torch
# MLP Model
class MLP(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim_1, hidden_dim_2=0):
        super(MLP, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim_1 = hidden_dim_1
        self.fc1 = torch.nn.Linear(self.input_dim, self.hidden_dim_1)
        self.fc1.weight.data.copy_(torch.eye(self.hidden_dim_1)) # weight initialization by identity matrix
        self.fc1.bias.data.fill_(0) # weight initialization by zero
        self.dropout = torch.nn.Dropout(p=0.2)
        if hidden_dim_2!=0:
            self.hidden_dim_2 = hidden_dim_2
            self.fc2 = torch.nn.Linear(self.hidden_dim_1, self.hidden_dim_2)
        else:self.hidden_dim_2 = None
    def forward(self, x):
        if self.hidden_dim_2 != None:
            if config.activation_function == 'relu':
                X_hidden = F.relu(self.fc2(self.fc1(x)))
            elif config.activation_function == 'selu':
                X_hidden = F.relu(self.fc2(self.fc1(x)))
        else:
            if config.activation_function == 'relu':
                # X_hidden = self.dropout(F.selu(self.fc1(x))) # relu
                X_hidden = torch.relu(self.fc1(x))  # relu
                # X_hidden = self.fc1(x)
            elif config.activation_function == 'selu':
                X_hidden = torch.selu(self.fc1(x))  # relu

        return X_hidden
mlp_model = MLP(config.input_dim, config.hidden_dim_1, config.hidden_dim_2)

In [3]:
for name, param in mlp_model.named_parameters():
    if param.requires_grad:
        print(name)
#         print(name, param.data)


fc1.weight
fc1.bias


In [4]:
if config.mlp_optimizer=='sgd':
    optimizer = torch.optim.SGD(mlp_model.parameters(), lr=config.learning_rate_mlp, momentum=config.sgd_momentum,
                                     dampening=config.sgd_dampening, weight_decay=config.sgd_weight_decay, nesterov=config.sgd_nesterov) # momentum=0.9, weight_decay=0, nesterov=false
elif config.mlp_optimizer == 'adam':
    # self.optimizer = torch.optim.Adam(self.mlp_model.parameters(), lr=config.learning_rate_mlp)
    optimizer = torch.optim.Adam(mlp_model.parameters(), lr=config.learning_rate_mlp,
                                      weight_decay=config.adam_weight_decay, amsgrad=config.adam_amsgrad)

In [5]:
# other variables
SMALL = float('-inf')
TASK_num = Y_train.shape[0]
TRAIN_size = Y_train.shape[1]

if config.using_mlp:
    if config.hidden_dim_2 != 0:
        d = config.hidden_dim_2
    else:
        d = config.hidden_dim_1
else:
    d = config.input_dim
with torch.no_grad():
    if config.w_from_matlab:
        W_mat = scipy.io.loadmat(config.train_weight_location)
        W = torch.tensor(W_mat['W']).float()
    else:
        W = torch.nn.Parameter(
            torch.empty(d, TASK_num).normal_(mean=0, std=0.1))  # default: 0.1 # compare this with matlab

    values = Y_train.unique()
    k = values.shape[0]
    pi = torch.nn.Parameter(torch.ones(k - 1, 1) * 1 / k)
    cum_pi = torch.cumsum(pi, dim=0)
    T = torch.log(cum_pi / (1 - cum_pi))
    T = T.repeat(1, TASK_num)

In [9]:
type(TASK_num)

int

In [6]:
# training
for adi in range(1):
    for t in range(2):
        print(adi, t)
        loss = 0
        if config.using_mlp:
            optimizer.zero_grad() # optimizer
            X_hidden_init = mlp_model(X_train)
        for i in range(TASK_num):
            for j in range(TRAIN_size):
                X_hidden = X_hidden_init[i,j,:]
                if Y_train[i,j]==k:
                    sig1=1
                    # sig2= logsig(T[int(Y_train[i,j])-2,i] - torch.matmul(X_hidden,W[:,i])) # logsig result is not same as matlab
                    sig2= 1 / (1 + torch.exp(-(T[int(Y_train[i,j])-2,i] - torch.matmul(X_hidden,W[:,i]))))
                    # sig2 = 1 / (1 + torch.exp(-(T[int(Y_train[i, j]) - 2, i] + torch.matmul(X_hidden, W[:, i])))) # Bug warning: according to eq1 it may be +
                    dif=SMALL # default: SMALL (Bug Warning: Not sure if it is BIG or not)
                elif Y_train[i,j]==1:
                    # sig1= logsig(T[int(Y_train[i,j])-1,i] - torch.matmul(X_hidden,W[:,i])) # Bug Warning: why not just sigmoid? Equ 1 paper
                    sig1= 1 / (1 + torch.exp(-(T[int(Y_train[i,j])-1,i] - torch.matmul(X_hidden,W[:,i]))))
                    sig2=0
                    dif=SMALL
                else:
                    # sig1= logsig2(T[int(Y_train[i,j])-1,i] - torch.matmul(X_hidden,W[:,i]))
                    sig1= 1 / (1 + torch.exp(-(T[int(Y_train[i,j])-1,i] - torch.matmul(X_hidden,W[:,i]))))
                    # sig1 = 1 / (1 + torch.exp(-(T[int(Y_train[i, j]) - 1, i] + torch.matmul(X_hidden, W[:, i])))) # Bug warning: according to eq1 it may be +
                    # sig2= logsig2(T[int(Y_train[i,j])-2,i] - torch.matmul(X_hidden,W[:,i]))
                    sig2= 1 / (1 + torch.exp(-(T[int(Y_train[i,j])-2,i] - torch.matmul(X_hidden,W[:,i]))))
                    # sig2 = 1 / (1 + torch.exp(-(T[int(Y_train[i, j]) - 2, i] + torch.matmul(X_hidden, W[:, i])))) # Bug warning: according to eq1 it may be +
                    dif= (T[int(Y_train[i,j])-2,i]-T[int(Y_train[i,j])-1,i])
                # loss

                l_cum=-torch.log(sig1-sig2)
                loss=loss+l_cum
#                 print(loss)
        loss.backward()
        optimizer.step()

0 0
0 1


In [7]:
for name, param in mlp_model.named_parameters():
    if param.requires_grad:
#         print(name)
        print(name, param.data)

fc1.weight tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [0., 1., 0.,  ..., 0., 0., 0.],
        [0., 0., 1.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 1., 0., 0.],
        [0., 0., 0.,  ..., 0., 1., 0.],
        [0., 0., 0.,  ..., 0., 0., 1.]])
fc1.bias tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.

In [8]:
sum(mlp_model(X_train)-X_train)

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], grad_fn=<AddBackward0>)

In [10]:
# training
for adi in range(1):
    for t in range(2):
        print(adi, t)
        loss = 0
        if config.using_mlp:
            optimizer.zero_grad() # optimizer
#             X_hidden_init = mlp_model(X_train)
        for i in range(TASK_num):
            for j in range(TRAIN_size):
#                 X_hidden = X_hidden_init[i,j,:]
                X_hidden = mlp_model(X_train[i,j,:])
                if Y_train[i,j]==k:
                    sig1=1
                    # sig2= logsig(T[int(Y_train[i,j])-2,i] - torch.matmul(X_hidden,W[:,i])) # logsig result is not same as matlab
                    sig2= 1 / (1 + torch.exp(-(T[int(Y_train[i,j])-2,i] - torch.matmul(X_hidden,W[:,i]))))
                    # sig2 = 1 / (1 + torch.exp(-(T[int(Y_train[i, j]) - 2, i] + torch.matmul(X_hidden, W[:, i])))) # Bug warning: according to eq1 it may be +
                    dif=SMALL # default: SMALL (Bug Warning: Not sure if it is BIG or not)
                elif Y_train[i,j]==1:
                    # sig1= logsig(T[int(Y_train[i,j])-1,i] - torch.matmul(X_hidden,W[:,i])) # Bug Warning: why not just sigmoid? Equ 1 paper
                    sig1= 1 / (1 + torch.exp(-(T[int(Y_train[i,j])-1,i] - torch.matmul(X_hidden,W[:,i]))))
                    sig2=0
                    dif=SMALL
                else:
                    # sig1= logsig2(T[int(Y_train[i,j])-1,i] - torch.matmul(X_hidden,W[:,i]))
                    sig1= 1 / (1 + torch.exp(-(T[int(Y_train[i,j])-1,i] - torch.matmul(X_hidden,W[:,i]))))
                    # sig1 = 1 / (1 + torch.exp(-(T[int(Y_train[i, j]) - 1, i] + torch.matmul(X_hidden, W[:, i])))) # Bug warning: according to eq1 it may be +
                    # sig2= logsig2(T[int(Y_train[i,j])-2,i] - torch.matmul(X_hidden,W[:,i]))
                    sig2= 1 / (1 + torch.exp(-(T[int(Y_train[i,j])-2,i] - torch.matmul(X_hidden,W[:,i]))))
                    # sig2 = 1 / (1 + torch.exp(-(T[int(Y_train[i, j]) - 2, i] + torch.matmul(X_hidden, W[:, i])))) # Bug warning: according to eq1 it may be +
                    dif= (T[int(Y_train[i,j])-2,i]-T[int(Y_train[i,j])-1,i])
                # loss

                l_cum=-torch.log(sig1-sig2)
                loss=loss+l_cum
#                 print(loss)
        loss.backward()
        optimizer.step()

0 0
0 1


In [11]:
for name, param in mlp_model.named_parameters():
    if param.requires_grad:
#         print(name)
        print(name, param.data)

fc1.weight tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [0., 1., 0.,  ..., 0., 0., 0.],
        [0., 0., 1.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 1., 0., 0.],
        [0., 0., 0.,  ..., 0., 1., 0.],
        [0., 0., 0.,  ..., 0., 0., 1.]])
fc1.bias tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.

In [12]:
sum(mlp_model(X_train)-X_train)

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], grad_fn=<AddBackward0>)